# 3rd Level Model Structure: Single Stage Reactive Distillation

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))

import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets

In [2]:
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule

from utility.display_utility import trans_product_mole, trans_product_mass, beautify2
from utility.model_utility import add_dual, update_dual, check_DOF
from utility.data_utility import cal_cnumber

model = pe.ConcreteModel()

# Global Set

In [3]:
model.TRAY = pe.RangeSet(1,5)

# Construct Reactive Stages

In [4]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].T_F
| reactive[1].P
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].T
| reactive[1].H_F
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].r_total_comp
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_block.r_WGS
| reactive[1].kinetics_bloc

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[5].VLE_block.n_ave
| reactive[5].VLE_block.n_ave_cal
| reactive[5].VLE_block.Hen
| reactive[5].VLE_block.Hen0
| reactive[5].VLE_block.gamma
| reactive[5].VLE_block.P_sat
| reactive[5].VLE_block.P_sat_Y
| reactive[5].VLE_block.P_sat_dY_inf
| reactive[5].VLE_block.P_sat_dY0
| reactive[5].VLE_block.Hen_ref
| reactive[5].VLE_block.Hen0_ref
| reactive[5].VLE_block.gamma_ref
| reactive[5].VLE_block.V_L
| reactive[5].VLE_block.V_L_dY_inf
| reactive[5].VLE_block.V_L_dY0
| reactive[5].VLE_block.poynting
--------------------------------------------------



# Construct a single condenser

In [5]:
model.condenser = pe.Block(rule=condenser_stage_rule)

| Importing Condenser Stage......
| Adding the following local variable:
------------------------------------
| condenser.T
| condenser.T_F
| condenser.P
| condenser.Q_main
| condenser.x_
| condenser.y_
| condenser.x
| condenser.y
| condenser.z
| condenser.L
| condenser.W
| condenser.V
| condenser.F
| condenser.H_L_
| condenser.H_V_
| condenser.H_L
| condenser.H_V
| condenser.H_F
| condenser.f_V
| condenser.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.energy_block.dH_F
| condenser.energy_block.dH_V
| condenser.energy_block.dH_L
| condenser.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.VLE_block.n_ave
| condenser.VLE_block.n_ave_cal
| condenser.VLE_block.Hen
| condenser.VLE_block.Hen0
| condenser.VLE_blo

# Linking Stage Variables

### Vapor Between Reactive Stages

In [6]:
def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

### Liquid Between Reactive Stages

In [7]:
def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

### Condenser

In [8]:
def V_condenser_rule(model):
    return model.reactive[model.TRAY.first()].V['out'] == model.condenser.V['in']
model.V_condenser_con = pe.Constraint(rule=V_condenser_rule)

def Vy_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].y[i] == model.condenser.y_['in',i]
model.Vy_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_condenser_rule)

def Vh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_V == model.condenser.H_V_['in']
model.Vh_condenser_con = pe.Constraint(rule=Vh_condenser_rule)

In [9]:
def L_condenser_rule(model):
    return model.reactive[model.TRAY.first()].L['in'] == model.condenser.L['out']
model.L_condenser_con = pe.Constraint(rule=L_condenser_rule)

def Lx_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].x_['in',i] == model.condenser.x[i]
model.Lx_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_condenser_rule)

def Lh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_L_['in'] == model.condenser.H_L
model.Lh_condenser_con = pe.Constraint(rule=Lh_condenser_rule)

In [10]:
model.obj = pe.Objective(expr = sum(model.reactive[j].T for j in model.TRAY) ,sense=pe.maximize)

# Load from single stage solutions

In [11]:
with open('../saved_solutions/1_stage_condenser_adiabatic.pickle', 'rb') as f:
    results_imported = pickle.load(f)
results_changed = results_imported

### Duplicate variable solution and bounds multiplier

In [12]:
for i in list(results_imported.Solution.Variable.keys()):
    if i.startswith('reactive[1].'):
        for j in model.TRAY:
            if j != 1:
                results_changed.Solution.Variable[i.replace('reactive[1].','reactive[{}].'.format(j))] = \
                results_imported.Solution.Variable[i]

### Duplicate constraint multiplier

In [13]:
for i in list(results_changed.Solution.Constraint.keys()):
    if i.startswith('reactive[1].'):
        for j in model.TRAY:
            if j != 1:
                results_changed.Solution.Constraint[i.replace('reactive[1].','reactive[{}].'.format(j))] = \
                results_imported.Solution.Constraint[i]

### Load changed solution into current model

In [14]:
model.solutions.load_from(results_changed)

# Fixing Redundent Stream Variables

In [15]:
# condenser
model.condenser.VLE_block.n_ave.fix(4)

model.condenser.F.fix(0)
model.condenser.T_F.fix(300+273.15)
model.condenser.z.fix(0)

model.condenser.V['P'].fix(0)
model.condenser.L['in'].fix(0)
for i in m.COMP_TOTAL: model.condenser.x_['in',i].fix(0)
model.condenser.H_L_['in'].fix(0)

In [16]:
# 'reboiler' fixing last stage V_in

model.reactive[model.TRAY.last()].V['in'].fix(0)
for i in m.COMP_TOTAL: model.reactive[model.TRAY.last()].y_['in',i].fix(0)
model.reactive[model.TRAY.last()].H_V_['in'].fix(0)

# Load Operating Parameters

In [17]:
# condenser
model.condenser.P.fix(19)
# model.condenser.T.fix(30+273.15)
model.condenser.T.fix(30+273.15)
model.condenser.L['out'].fix(0)

# reactive stage
for j in model.reactive:
    model.reactive[j].cat.fix(5000)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(20)
    
    model.reactive[j].F.fix(1)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(1/(1+2))
    model.reactive[j].z['H2'].fix(2/(1+2))
    model.reactive[j].z['C30H62'].fix(0)
    model.reactive[j].V['P'].fix(0)
    model.reactive[j].L['P'].fix(0)
    # model.reactive[j].Q_main.fix(0)
    # model.reactive[j].T.setub(240+273.15)
model.reactive[1].T.setub(220+273.15)
model.reactive[2].T.setub(225+273.15)
model.reactive[3].T.setub(230+273.15)
model.reactive[4].T.setub(235+273.15)
model.reactive[5].T.setub(240+273.15)

In [18]:
check_DOF(pe,model)

Active Equality Constraints:	 7672
Active Inequality Constraints:	 0
Active Variables:		 7899
Fixed Variables:		 222
DOF:				 5


In [19]:
opt = pe.SolverFactory('ipopt')
opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'
# opt.options['nlp_scaling_method'] = None
# opt.options['constr_viol_tol'] = 1e-7
# opt.options['acceptable_constr_viol_tol'] = 1e-7
opt.options['max_iter'] = 7000
# opt.options['dual_inf_tol'] = '+inf'
# opt.options['acceptable_dual_inf_tol'] = '+inf'

In [20]:
add_dual(pe,model)
update_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


In [21]:
results = opt.solve(model,tee=True)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
max_iter=7000


List of user-set options:

                                    Name   Value                used
                           linear_solver = ma86                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver ma86.

Number of nonzeros in equality constraint Jacobian...:    25272
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.....

  74 -2.5157500e+03 8.96e+03 6.77e+05  -1.0 9.01e+03  -2.1 6.07e-04 5.18e-03h  1
  75 -2.5157500e+03 8.96e+03 6.77e+05  -1.0 1.04e+04  -2.5 6.13e-04 5.21e-05h  1
  76 -2.5157500e+03 8.93e+03 6.75e+05  -1.0 8.96e+03  -2.1 8.94e-04 4.00e-03f  1
  77 -2.5157500e+03 8.93e+03 6.75e+05  -1.0 1.85e+04  -2.6 4.13e-04 4.88e-05h  1
  78 -2.5157500e+03 8.90e+03 6.73e+05  -1.0 8.93e+03  -2.2 1.39e-03 2.76e-03h  1
  79 -2.5157500e+03 8.79e+03 6.65e+05  -1.0 8.90e+03  -1.7 1.16e-02 1.23e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80 -2.5157500e+03 8.78e+03 6.64e+05  -1.0 8.79e+03  -2.2 2.38e-03 1.18e-03h  1
  81 -2.5157500e+03 8.71e+03 6.59e+05  -1.0 8.78e+03  -1.8 5.78e-03 7.72e-03h  1
  82 -2.5157500e+03 8.71e+03 6.58e+05  -1.0 8.71e+03  -2.3 1.75e-03 6.63e-04h  1
  83 -2.5157500e+03 8.71e+03 6.58e+05  -1.0 1.63e+05  -2.7 5.19e-05 1.94e-05h  1
  84 -2.5157500e+03 8.70e+03 6.58e+05  -1.0 8.71e+03  -2.3 1.29e-03 7.74e-04h  1
  85 -2.5157500e+03 8.70e+03

 167 -2.5157467e+03 9.15e+02 6.45e+05  -1.0 9.47e+02    -  2.42e-02 3.35e-02h  1
 168 -2.5157465e+03 8.75e+02 1.12e+06  -1.0 9.15e+02    -  2.54e-02 4.36e-02h  1
 169 -2.5157464e+03 8.52e+02 1.07e+06  -1.0 8.76e+02    -  1.21e-02 2.66e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170 -2.5157464e+03 8.48e+02 1.07e+06  -1.0 8.52e+02    -  3.46e-02 5.01e-03h  1
 171 -2.5157463e+03 8.30e+02 1.09e+06  -1.0 8.48e+02    -  6.11e-04 2.07e-02h  1
 172 -2.5157462e+03 8.14e+02 9.61e+05  -1.0 1.41e+03    -  7.74e-03 1.99e-02h  1
 173 -2.5157462e+03 8.14e+02 9.61e+05  -1.0 4.34e+03    -  1.64e-03 3.34e-04h  1
 174 -2.5157473e+03 8.13e+02 9.61e+05  -1.0 1.98e+01   6.0 4.96e-04 6.96e-05h  1
 175r-2.5157473e+03 8.13e+02 1.00e+03   2.6 0.00e+00    -  0.00e+00 3.67e-07R  4
 176r-2.5157471e+03 8.10e+02 8.85e+03   2.6 2.97e+05    -  1.77e-04 1.01e-05f  1
 177r-2.5157475e+03 8.85e+01 9.03e+03   0.5 2.91e+05    -  4.58e-03 2.66e-03f  1
 178 -2.5157475e+03 8.85e+01

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 260 -2.5157500e+03 6.84e+00 1.87e+08  -1.0 2.48e+00   4.8 1.89e-03 1.29e-02f  1
 261 -2.5157500e+03 4.85e+00 4.67e+08  -1.0 1.12e+00   4.3 3.94e-02 7.79e-02h  1
 262 -2.5157500e+03 3.85e+00 4.83e+08  -1.0 6.90e-01   3.8 8.97e-01 1.76e-01h  1
 263 -2.5157500e+03 3.47e+00 6.00e+08  -1.0 5.69e-01   3.3 7.16e-01 9.33e-02h  1
 264 -2.5157500e+03 2.17e+00 4.11e+08  -1.0 5.16e-01   3.8 4.40e-01 3.54e-01h  1
 265 -2.5157500e+03 1.41e+00 2.98e+08  -1.0 3.33e-01   3.3 4.68e-01 3.56e-01h  1
 266 -2.5157500e+03 1.23e+00 2.39e+08  -1.0 6.82e-01   2.8 4.71e-02 1.29e-01h  1
 267 -2.5157500e+03 4.62e-01 1.54e+07  -1.0 1.87e-01   3.2 2.63e-01 6.33e-01h  1
 268 -2.5157500e+03 3.72e-01 1.68e+08  -1.0 6.85e-02   4.6 9.63e-01 1.91e-01h  1
 269 -2.5157500e+03 3.61e-01 1.39e+08  -1.0 5.56e-02   4.1 1.00e+00 3.10e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 270 -2.5157500e+03 1.46e+00

 352 -2.5154926e+03 6.97e-02 3.21e+03  -1.7 1.09e+00  -0.8 1.00e+00 1.00e+00H  1
 353 -2.5154956e+03 7.03e-02 6.23e+03  -1.7 1.49e+02    -  6.64e-01 5.46e-03h  7
 354 -2.5155200e+03 6.14e-02 8.62e+03  -1.7 7.81e+01    -  1.00e+00 1.10e-01f  4
 355 -2.5155223e+03 6.20e-02 9.13e+03  -1.7 5.01e+01    -  6.39e-01 1.56e-02h  7
 356 -2.5155234e+03 6.23e-02 9.59e+03  -1.7 4.94e+01    -  1.00e+00 7.81e-03h  8
 357 -2.5155245e+03 6.19e-02 9.60e+03  -1.7 5.00e+01    -  1.00e+00 7.81e-03h  8
 358 -2.5155256e+03 6.17e-02 9.59e+03  -1.7 4.88e+01    -  1.00e+00 7.81e-03h  8
 359 -2.5155256e+03 6.17e-02 9.65e+03  -1.7 4.89e+01    -  1.00e+00 1.53e-05h 17
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 360 -2.5156672e+03 2.88e+00 3.32e+03  -1.7 4.85e+01    -  1.00e+00 1.00e+00s 22
 361 -2.5156653e+03 6.27e+00 7.49e+02  -1.7 4.68e+00    -  1.00e+00 1.00e+00s 22
 362 -2.5156648e+03 4.72e-01 2.77e+01  -1.7 4.20e+00    -  1.00e+00 1.00e+00s 22
 363 -2.5156647e+03 1.44e-02

 445 -2.5157500e+03 7.96e-09 1.02e+01  -8.6 1.00e-02    -  8.43e-01 9.70e-01h  1
 446 -2.5157500e+03 6.63e-09 2.68e+01  -8.6 8.32e-03    -  5.47e-01 1.00e+00f  1
 447 -2.5157500e+03 5.85e-09 5.64e+01  -8.6 4.65e-03    -  3.25e-01 1.00e+00h  1
 448 -2.5157500e+03 4.62e-09 6.58e+01  -8.6 1.26e-03    -  2.20e-01 1.00e+00h  1
 449 -2.5157500e+03 4.76e-09 6.53e+00  -8.6 2.50e-04    -  3.61e-01 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 450 -2.5157500e+03 3.15e-09 2.92e+00  -8.6 8.58e-05    -  2.24e-01 1.00e+00h  1
 451 -2.5157500e+03 8.13e-09 1.52e+00  -8.6 2.16e-04    -  5.54e-01 1.00e+00h  1
 452 -2.5157500e+03 2.62e-09 1.59e+00  -8.6 1.55e-04    -  3.43e-01 1.00e+00h  1
 453 -2.5157500e+03 4.77e-09 4.71e-01  -8.6 1.54e-04    -  7.59e-01 1.00e+00h  1
 454 -2.5157500e+03 1.96e-09 2.11e-07  -8.6 1.00e-04    -  1.00e+00 1.00e+00h  1
 455 -2.5157500e+03 1.43e-09 3.39e-01  -8.6 2.61e-05    -  3.56e-01 1.00e+00h  1
 456 -2.5157500e+03 1.42e-09

In [22]:
update_dual(pe,model)

In [23]:
beautify2(pe,model)

Here comes the result:
----------------------------------------------------------------------------------------------------
		T		 Q			 V			 L
               30.00 	       -78.4611827574 	         0.8086594835 	         0.0000000000
              220.00 	       -45.5772525035 	         2.1845354469 	         0.0094784672
              225.00 	       -45.9444348179 	         1.7378312110 	         0.0168938726
              230.00 	       -46.1100799358 	         1.2983778778 	         0.0218616813
              235.00 	       -46.1740540679 	         0.8632509010 	         0.0243023708
              240.00 	       -46.2016415083 	         0.4307500024 	         0.0244489846
----------------------------------------------------------------------------------------------------
Top
V	 0.8086594835099002
L	 0.11058561661141836
W	 1.2652903276995513
----------------------------------------------------------------------------------------------------
Bottom
L	 0.024448984640696027
--------------

In [24]:
model.reactive[1].kinetics_block.r_FT_total.value

0.28040337507132407

In [25]:
# model.solutions.store_to(results)
# with open('../saved_solutions/3_stage_condenser_240C.pickle','wb') as f:
#     pickle.dump(results,f)

In [26]:
model.condenser.L['out'].fixed = False
for j in model.reactive:
    model.reactive[j].T.fixed=True

In [27]:
model.del_component(model.obj)
model.obj = pe.Objective(expr = model.condenser.L['out'],sense=pe.maximize)

# So, what exactly does adding a reflux do?

In [28]:
Refluxrange = np.linspace(0,2,21)

In [29]:
cd_data = {};
cd_data['Re'] = []; cd_data['D'] = []; cd_data['V'] = []
cd_data['x'] = {}; cd_data['y'] = {}; cd_data['g'] = {}; cd_data['d'] = {};
for i in m.COMP_TOTAL:
    cd_data['x'][i] = []
    cd_data['y'][i] = []
    cd_data['g'][i] = []
    cd_data['d'][i] = []
    
rf_data = {}
for j in model.reactive:
    rf_data[j] = {}
    rf_data[j]['r'] = {}; rf_data[j]['b'] = {}; rf_data[j]['x'] = {};rf_data[j]['y'] = {};
    rf_data[j]['T'] = []; rf_data[j]['Q'] = []; rf_data[j]['V'] = []; rf_data[j]['L'] = []; 
    rf_data[j]['r_WGS'] = []; rf_data[j]['r_FT'] = []
    for i in m.COMP_TOTAL:
        rf_data[j]['r'][i] = []
        rf_data[j]['b'][i] = []
        rf_data[j]['x'][i] = []
        rf_data[j]['y'][i] = []       

In [30]:
for re in Refluxrange:
    # model.condenser.L['out'].fixed = False
    model.condenser.L['out'].setub(re)
    # model.reactive[1].T.fix(t)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    print('Solved\t|Reflux = {:.3f} kmol/s\t|Vapor = {:.3f} kmol/s\t|Distillate = {:.3f} kmol/s\t|Bottom = {:.3f} kmol/s'.\
          format(model.condenser.L['out'].value,model.condenser.V['out'].value,model.condenser.L['P'].value,model.reactive[3].L['out'].value))

    cd_data['V'].append(model.condenser.V['out'].value)
    cd_data['D'].append(model.condenser.L['P'].value)
    cd_data['Re'].append(model.condenser.L['out'].value)

    for i in model.reactive[1].r_total_comp:
        cd_data['x'][i].append(model.condenser.x[i].value)
        cd_data['y'][i].append(model.condenser.y[i].value)
        cd_data['g'][i].append(model.condenser.y[i].value*model.condenser.V['out'].value)
        cd_data['d'][i].append(model.condenser.x[i].value*model.condenser.L['P'].value)
    
    for j in model.reactive:      
        rf_data[j]['T'].append(model.reactive[j].T.value)
        rf_data[j]['Q'].append(model.reactive[j].Q_main.value)
        rf_data[j]['V'].append(model.reactive[j].V['out'].value)
        rf_data[j]['L'].append(model.reactive[j].L['out'].value)
        rf_data[j]['r_WGS'].append(model.reactive[j].kinetics_block.r_WGS.value)
        rf_data[j]['r_FT'].append(model.reactive[j].kinetics_block.r_FT_total.value)

        for i in model.reactive[1].r_total_comp:
            rf_data[j]['r'][i].append(model.reactive[j].r_total_comp[i].value)
            rf_data[j]['b'][i].append(model.reactive[j].x[i].value*model.reactive[j].L['out'].value)
            rf_data[j]['x'][i].append(model.reactive[j].x[i].value)
            rf_data[j]['y'][i].append(model.reactive[j].y[i].value)

Solved	|Reflux = 0.000 kmol/s	|Vapor = 0.809 kmol/s	|Distillate = 0.111 kmol/s	|Bottom = 0.022 kmol/s
Solved	|Reflux = 0.100 kmol/s	|Vapor = 0.816 kmol/s	|Distillate = 0.105 kmol/s	|Bottom = 0.028 kmol/s
Solved	|Reflux = 0.200 kmol/s	|Vapor = 0.823 kmol/s	|Distillate = 0.101 kmol/s	|Bottom = 0.034 kmol/s
Solved	|Reflux = 0.300 kmol/s	|Vapor = 0.830 kmol/s	|Distillate = 0.096 kmol/s	|Bottom = 0.041 kmol/s
Solved	|Reflux = 0.400 kmol/s	|Vapor = 0.837 kmol/s	|Distillate = 0.093 kmol/s	|Bottom = 0.048 kmol/s
Solved	|Reflux = 0.500 kmol/s	|Vapor = 0.845 kmol/s	|Distillate = 0.089 kmol/s	|Bottom = 0.054 kmol/s
Solved	|Reflux = 0.600 kmol/s	|Vapor = 0.852 kmol/s	|Distillate = 0.086 kmol/s	|Bottom = 0.061 kmol/s
Solved	|Reflux = 0.700 kmol/s	|Vapor = 0.859 kmol/s	|Distillate = 0.083 kmol/s	|Bottom = 0.067 kmol/s
Solved	|Reflux = 0.800 kmol/s	|Vapor = 0.866 kmol/s	|Distillate = 0.081 kmol/s	|Bottom = 0.073 kmol/s
Solved	|Reflux = 0.900 kmol/s	|Vapor = 0.874 kmol/s	|Distillate = 0.078 kmol/s	|Bo

In [31]:
cnumber_range = range(1,57)

In [32]:
def trans_cnumber(dic):
    molefraction = {}
    for i in range(1,57):
        molefraction[i] = []
    for i in m.COMP_ORG:
        molefraction[cal_cnumber(i)].append(np.array(dic[i]))
    for i in range(1,57):
        molefraction[i] = np.sum(molefraction[i],0)
    length = len(molefraction[1])
    tmp = {}
    for j in range(length):
        tmp[j] = []
        for i in range(1,57):
            tmp[j].append(molefraction[i][j])
    return tmp

In [33]:
g_data = trans_cnumber(cd_data['g'])
d_data = trans_cnumber(cd_data['d'])
b_data = trans_cnumber(rf_data[3]['b'])
# r_data = trans_cnumber(rf_data_master[t]['r'])

cd_x_data = trans_cnumber(cd_data['x'])
rf_x_data = {}
for j in model.reactive:
    rf_x_data[j] = trans_cnumber(rf_data[j]['x'])

In [36]:
 def plot_distribution(index):
    fig, (ax,ax2) = plt.subplots(2,1,figsize=(16,12))
    ax.plot(cnumber_range,g_data[index],'co-')
    ax.plot(cnumber_range,d_data[index],'go-')
    ax.plot(cnumber_range,b_data[index],'ro-')
    ax.set_yscale("log")
    ax.set_ylim(1e-12, 1)
    ax.legend(['Vapor','Distillate','Bottom'],fontsize=18)
    ax.set_title('T, Reflux {:.2f} kmol/s'.format(cd_data['Re'][index]),fontsize=18)

    ax.set_ylabel('Molar Flow (kmol/s)', color='K',fontsize=18)
    ax.set_xlabel('Carbon Number', color='K',fontsize=18)
    # ax.tick_params('y', colors='k',labelsize=18)
    # ax.tick_params('x', colors='k',labelsize=18)

    ax2.plot(cnumber_range,cd_x_data[index],'go-')
    ax2.plot(cnumber_range,rf_x_data[1][index],'co-')
    ax2.plot(cnumber_range,rf_x_data[2][index],'bo-')
    ax2.plot(cnumber_range,rf_x_data[3][index],'mo-')
    ax2.plot(cnumber_range,rf_x_data[4][index],'ko-')
    ax2.plot(cnumber_range,rf_x_data[5][index],'ro-')

    ax2.legend(['Condenser','Stage 1','Stage 2','Stage 3','Stage 4','Stage 5'],fontsize=18)
    ax2.set_title('Liquid Composition (Mole)',fontsize=18)

    ax2.set_ylabel('Molar Fraction', color='K',fontsize=18)
    ax2.set_xlabel('Carbon Number', color='K',fontsize=18)

    ax.grid()
    ax2.grid()
    plt.show()

In [37]:
widgets.interact(plot_distribution, index = widgets.IntSlider(
    value=0,
    min=0,
    max=20,
    step=1,
    description='Reflux:',)
);

interactive(children=(IntSlider(value=0, description='Reflux:', max=20), Output()), _dom_classes=('widget-inte…